Import modules

In [10]:
from keras.models import Sequential, load_model, Model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Input
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import cv2

%matplotlib inline

Load data.

In [18]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)

from bounding_boxes import *
from data_generators import get_data_with_bbox_coords
import dataset
from callbacks import ShowSegmentation

bboxes = get_bounding_boxes('../data')
bboxes = largest_bbox_per_image(bboxes)
train_data = dataset.train
val_data = dataset.validation
 
train_gen = get_data_with_bbox_coords(train_data, '../data/train', bboxes)
val_gen = get_data_with_bbox_coords(val_data, '../data/train', bboxes)
n_labels = len(dataset.labels)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
def binning_generator(data_gen, n_bins = 10):
    #
    for images, labels in data_gen:
        batch_size = len(images)
        label_x1 = np.zeros((batch_size, n_bins))
        label_x2 = np.zeros((batch_size, n_bins))
        label_y1 = np.zeros((batch_size, n_bins))
        label_y2 = np.zeros((batch_size, n_bins))
        for i,l, x1, x2, y1, y2 in zip(images, labels, label_x1, label_x2, label_y1, label_y2):
            shape = np.shape(i)
            bin_size = np.ceil(shape[1] / float(n_bins))
            t0 = l[0]
            t1 = l[1]
            l[0] = l[0] // bin_size
            l[1] = l[1] // bin_size
            l[2] = (l[2]+t0) // bin_size
            l[3] = (l[3]+t1) // bin_size
            x1[l[0]] = 1 
            x2[l[1]] = 1 
            y1[l[2]] = 1 
            y2[l[3]] = 1 
        yield images, [label_x1, label_x2, label_y1, label_y2]

Construct model 
Todo: softmax for binning training

In [49]:
def keras_model_conv():
    input1 = Input(shape=(3, 256, 256))
    layer = BatchNormalization(axis=1,  input_shape=(3, 256, 256))(input1)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Dropout(0.50)(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Dropout(0.50)(layer)
    layer = Convolution2D(16, 3, 3, border_mode='same', activation='relu')(layer)
    layer = MaxPooling2D(pool_size=(2,2))(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    output1 = Dense(10, activation='softmax')(layer)
    output2 = Dense(10, activation='softmax')(layer)
    output3 = Dense(10, activation='softmax')(layer)
    output4 = Dense(10, activation='softmax')(layer)
    model = Model(input=input1, output=[output1, output2, output3, output4])
    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

    return model

In [50]:
model = keras_model_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 3, 256, 256)   0                                            
____________________________________________________________________________________________________
batchnormalization_12 (BatchNorm (None, 3, 256, 256)   12          input_12[0][0]                   
____________________________________________________________________________________________________
convolution2d_34 (Convolution2D) (None, 16, 256, 256)  448         batchnormalization_12[0][0]      
____________________________________________________________________________________________________
maxpooling2d_34 (MaxPooling2D)   (None, 16, 128, 128)  0           convolution2d_34[0][0]           
___________________________________________________________________________________________

In [51]:
model.fit_generator(binning_generator(train_gen), samples_per_epoch=len(train_data),
                    nb_epoch=15, verbose=1,
                    callbacks=None,
                    validation_data=binning_generator(val_gen), nb_val_samples=len(val_data))

Epoch 1/15
3008/3021 [============================>.] - ETA: 1s - loss: 12.2706 - dense_57_loss: 3.0588 - dense_58_loss: 3.2385 - dense_59_loss: 2.6219 - dense_60_loss: 3.3513 - dense_57_acc: 0.2124 - dense_58_acc: 0.2995 - dense_59_acc: 0.2749 - dense_60_acc: 0.2178  

/Users/pieter/anaconda2/envs/medim/lib/python2.7/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3040/3021 [==============================] - 501s - loss: 12.2164 - dense_57_loss: 3.0451 - dense_58_loss: 3.2235 - dense_59_loss: 2.6127 - dense_60_loss: 3.3351 - dense_57_acc: 0.2141 - dense_58_acc: 0.2997 - dense_59_acc: 0.2760 - dense_60_acc: 0.2188 - val_loss: 9.0106 - val_dense_57_loss: 2.2445 - val_dense_58_loss: 2.2561 - val_dense_59_loss: 2.2544 - val_dense_60_loss: 2.2556 - val_dense_57_acc: 0.1914 - val_dense_58_acc: 0.1576 - val_dense_59_acc: 0.3021 - val_dense_60_acc: 0.1068
Epoch 2/15
3040/3021 [==============================] - 471s - loss: 6.0360 - dense_57_loss: 1.5071 - dense_58_loss: 1.4719 - dense_59_loss: 1.5146 - dense_60_loss: 1.5424 - dense_57_acc: 0.4319 - dense_58_acc: 0.4707 - dense_59_acc: 0.4543 - dense_60_acc: 0.4355 - val_loss: 7.2417 - val_dense_57_loss: 1.7951 - val_dense_58_loss: 1.7909 - val_dense_59_loss: 1.8157 - val_dense_60_loss: 1.8400 - val_dense_57_acc: 0.5443 - val_dense_58_acc: 0.5378 - val_dense_59_acc: 0.5638 - val_dense_60_acc: 0.4518

KeyboardInterrupt: 